In [1]:
from sklearn import datasets
import numpy as np

# Import review data

In [8]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)
print "twenty_train.data.shape", np.array(twenty_train.data).shape

twenty_train.data.shape (11314,)


# CountVectorizer
which is important aspect for our learnvocabulary --> bag-of-words method

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
print "X_train_counts.shape", X_train_counts.shape
print "X_train_counts", X_train_counts
print count_vect.vocabulary_.get(u'algorithm')

X_train_counts.shape (11314, 130107)
X_train_counts   (0, 56979)	1
  (0, 43959)	1
  (0, 58175)	2
  (0, 32863)	3
  (0, 50527)	2
  (0, 111322)	1
  (0, 99721)	1
  (0, 43969)	5
  (0, 32576)	1
  (0, 79569)	1
  (0, 64186)	2
  (0, 29573)	1
  (0, 26656)	1
  (0, 7367)	1
  (0, 54006)	1
  (0, 90379)	1
  (0, 118983)	1
  (0, 89362)	1
  (0, 37413)	1
  (0, 76032)	1
  (0, 3411)	1
  (0, 87620)	1
  (0, 95162)	1
  (0, 64095)	1
  (0, 57980)	1
  :	:
  (11313, 39169)	1
  (11313, 26430)	3
  (11313, 42714)	1
  (11313, 125095)	1
  (11313, 25581)	3
  (11313, 5417)	1
  (11313, 62477)	1
  (11313, 71813)	1
  (11313, 50654)	1
  (11313, 20003)	1
  (11313, 67867)	1
  (11313, 118927)	1
  (11313, 40477)	1
  (11313, 100058)	1
  (11313, 50958)	1
  (11313, 92999)	1
  (11313, 25578)	1
  (11313, 81822)	1
  (11313, 26411)	3
  (11313, 119489)	1
  (11313, 78724)	2
  (11313, 74730)	4
  (11313, 105878)	1
  (11313, 53400)	1
  (11313, 34780)	3
27366


# Feed counted-vectors into TfidfTransformer
* So we will know 

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print "X_train_tf.shape", X_train_tf.shape
print X_train_tf[:1]

X_train_tf.shape (11314, 130107)
  (0, 56979)	0.065795169496
  (0, 43959)	0.065795169496
  (0, 58175)	0.131590338992
  (0, 32863)	0.197385508488
  (0, 50527)	0.131590338992
  (0, 111322)	0.065795169496
  (0, 99721)	0.065795169496
  (0, 43969)	0.32897584748
  (0, 32576)	0.065795169496
  (0, 79569)	0.065795169496
  (0, 64186)	0.131590338992
  (0, 29573)	0.065795169496
  (0, 26656)	0.065795169496
  (0, 7367)	0.065795169496
  (0, 54006)	0.065795169496
  (0, 90379)	0.065795169496
  (0, 118983)	0.065795169496
  (0, 89362)	0.065795169496
  (0, 37413)	0.065795169496
  (0, 76032)	0.065795169496
  (0, 3411)	0.065795169496
  (0, 87620)	0.065795169496
  (0, 95162)	0.065795169496
  (0, 64095)	0.065795169496
  (0, 57980)	0.065795169496
  :	:
  (0, 115475)	0.131590338992
  (0, 75004)	0.065795169496
  (0, 30044)	0.263180677984
  (0, 115701)	0.065795169496
  (0, 66608)	0.131590338992
  (0, 52907)	0.065795169496
  (0, 32976)	0.065795169496
  (0, 103024)	0.065795169496
  (0, 30142)	0.065795169496
  (0, 1

# Predict test data

In [5]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print "X_train_tfidf.shape", X_train_tfidf.shape

#Training a classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
print "clf", clf

docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
print "X_new_counts", X_new_counts
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
  print '%r => %s' % (doc, twenty_train.target_names[category])

X_train_tfidf.shape (11314, 130107)
clf MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
X_new_counts   (0, 59626)	1
  (0, 68532)	1
  (0, 76876)	1
  (1, 54467)	1
  (1, 59961)	1
  (1, 68532)	1
  (1, 89860)	1
  (1, 90045)	1
  (1, 114455)	1
'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => rec.autos


In [7]:
#Building a pipeline
#vectorizer => transformer => classifier
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()) ])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

import numpy as np
twenty_test = fetch_20newsgroups(subset='test',categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
print "naive bayes", np.mean(predicted == twenty_test.target)


from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])
_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted = text_clf.predict(docs_test)
print "SVM", np.mean(predicted == twenty_test.target)

naive bayes 0.77389803505
SVM 0.823818374934
